In [2]:
from keras.models import load_model
from utils import get_audio_config, extract_feature
from librosa import load
import numpy as np
non_image_features = ['contrast', 'rmse', 'mfcc', 'tonnetz']
model = load_model('RAMS_trial_2_HS')

def reply_emotion(user_emotion):
    os.system("say "+" are you feeling "+user_emotion)

import pyaudio
import os
import wave
from sys import byteorder
from array import array
from struct import pack

THRESHOLD = 500
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 16000

SILENCE = 30

def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)

            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    r = array('h', [0 for i in range(int(seconds*RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds*RATE))])
    return r

def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.

    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.5 seconds of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > SILENCE:
            break

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    r = normalize(r)
    r = trim(r)
    r = add_silence(r, 0.5)
    return sample_width, r

def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record()
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()





In [ ]:
model = load_model('RAMS_trial_2_HS')

In [19]:
def predict(audio_path):    
    CHUNK_LENGTH = 2048
    data, fs = load(audio_path)
    chunk_size = int(CHUNK_LENGTH * fs / 1000)
    num_chunks = int(len(data) // chunk_size)
    
    all_snips = []
    for i in range(num_chunks):
        snippet = data[i*chunk_size:i*chunk_size+chunk_size]
        all_snips.append(snippet)
    if (len(data) % chunk_size) > 0:
        snippet = data[num_chunks*chunk_size:]
        snippet = np.resize(snippet, chunk_size)
        all_snips.append(snippet)
    for snip in all_snips:
        image_audio_config = get_audio_config(['image'])
        flat_audio_config = get_audio_config(non_image_features)
        image_feature = extract_feature([snip, fs], predict=True, **image_audio_config).reshape(1, 128,  1412, 1)
        flat_feature = extract_feature([snip, fs], predict=True, **flat_audio_config).reshape(1,1, 54, 1)
        feature = [image_feature, flat_feature]
        prediction = model.predict(feature)
        emotion_class = np.argmax(prediction)
        if emotion_class == 0:
            emotion = 'happy'
            print("prediction: HAPPY")
        else:
            emotion = 'sad'
            print("prediction: SAD")
        print("happy confidence: " + str(prediction[0][0]*100) + "%")
        print("sad confidence: " + str(prediction[0][1]*100) + "%")
    return prediction, emotion

In [20]:

while True:
    os.system("say "+ "Please speak into the microphone")	
    print("Please speak into the microphone")
    record_to_file('emotion.wav')
    print("done")
    prediction = predict('emotion.wav')
    reply_emotion(prediction[1])


Please speak into the microphone
done
prediction: SAD
happy confidence: 43.51509213447571%
sad confidence: 56.484901905059814%
prediction: HAPPY
happy confidence: 55.85409998893738%
sad confidence: 44.14590299129486%
prediction: HAPPY
happy confidence: 53.3597469329834%
sad confidence: 46.64025008678436%
prediction: SAD
happy confidence: 14.005182683467865%
sad confidence: 85.9948217868805%
Please speak into the microphone
done
prediction: SAD
happy confidence: 30.687546730041504%
sad confidence: 69.3124532699585%
prediction: HAPPY
happy confidence: 85.1100504398346%
sad confidence: 14.889952540397644%
prediction: HAPPY
happy confidence: 88.29403519630432%
sad confidence: 11.705958098173141%
Please speak into the microphone


KeyboardInterrupt: 